In [ ]:
import sys
import glob
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

nc_path = '/Users/research/Documents/GitHub/gliderProcessing/glider_data/echo_20230113/delayed/nc/'
processing_mode = 'delayed'

files = sorted(glob.glob(nc_path + '*{}*.nc'.format(processing_mode)))
data_list = []
glider_record_list = []

for f in files:
    try:
        tmpData = xr.open_dataset(f, engine='netcdf4', decode_times=False)
        data_list.append(tmpData)
        tmpGliderRecordData = xr.open_dataset(f, engine='netcdf4', group='glider_record', decode_times=False)
        glider_record_list.append(tmpGliderRecordData)
        print('Processed {}'.format(f))
    except Exception as e:
        print('Ignore {}. Error: {}'.format(f, e))

raw_data = xr.concat(glider_record_list, dim='time').sortby('time')
data = xr.concat(data_list, dim='time').sortby('time')

# Convert xarray datasets to pandas dataframes
raw_data = raw_data.to_dataframe().reset_index()
data = data.to_dataframe().reset_index()

## Correct lon/lat dead reckoning

In [ ]:
# Path containing python functions
path = ['/Users/research/Documents/GitHub/gliderProcessing/glider_processing_scripts/']
sys.path.append(path[0])

from gliderfuncs import correct_dead_reckoning, findProfiles

data['lon_qc'],data['lat_qc'] = correct_dead_reckoning(raw_data['m_lon'],raw_data['m_lat'],raw_data['time'],raw_data['x_dr_state'],raw_data['m_gps_lon'],raw_data['m_gps_lat'])


## Find Profiles

In [ ]:
data['profile_index'],data['profile_direction']=findProfiles(data['time'],data['depth'],stall=20,shake=200)

## Data QC

## Attributes + Data Saving

In [ ]:
from data2attr import save_netcdf

In [ ]:
fname='echo-trajectory_file.nc'
gliders_db = '/Users/research/Documents/GitHub/gliderProcessing/glider_reference_information/glider_serial-numbers_and_sensor-serial-numbers.csv'
metadata_source = '/Users/research/Documents/GitHub/gliderProcessing/glider_data/echo_20230113/metadata.yml'
encoder = '/Users/research/Documents/GitHub/gliderProcessing/glider_processing_scripts/attributes/glider_dac_3.0_conventions.yml'

source_info = {
    'gliders_db': gliders_db,
    'metadata_source': metadata_source,
    'encoder': encoder,
    'processing_mode': 'delayed',
    'data_type': 'trajectory',
    'data_source': files,
    'filename': fname,
    'filepath': nc_path
}

save_netcdf(data, raw_data,source_info)

In [ ]:

traj_file_name = sorted(glob.glob(nc_path + '*{}*.nc'.format('trajectory')))
traj_data = xr.open_dataset(traj_file_name[0], engine='netcdf4', decode_times=False)

traj_data

## PLOTS

In [ ]:

fig, ax = plt.subplots(figsize=(14,7))
plt.plot(data['time'],data['depth'])
plt.gca().invert_yaxis()
ax.set(xlabel='time', ylabel='depth(m)',
       title='Glider Dives')


fig, ax = plt.subplots(figsize=(14,7))
idx_down = np.flatnonzero(data['profile_index'] == 1)
idx_up = np.flatnonzero(data['profile_index'] == 2)
ax.plot(data['time'].values[idx_down], data['depth'].values[idx_down], 'o', label='down cast')
ax.plot(data['time'].values[idx_up], data['depth'].values[idx_up], 'o', label='up cast')
ax.set(xlabel='time (s)', ylabel='depth (m)',
       title='Algorithm for Profile Splitting Code')
plt.gca().invert_yaxis()
ax.grid()
ax.legend()
plt.show()


fig, ax = plt.subplots(figsize=(14,7))
ax.plot(data['lon_qc'],data['lat_qc'],'o',label='corrected(underwater)')
ax.plot(raw_data['m_gps_lon'],raw_data['m_gps_lat'],'o',label='GPS(surface)')
ax.set(xlabel='longitude', ylabel='latitude',
       title='Algorithm for Deadreckoning Position Correction')
ax.grid()
ax.legend()
plt.show()

